# Excel2LaTeXViaPython

This code reads in an excel file, and writes a LaTeX file file containing a tabular environment that attempts to match the styling of the excel table as much as possible.

## User input
In the Jupyter cell below enter the name of the of the input Excel file, and the name of the tex file to save the outputed LaTeX code to

In [149]:
# When looping over sheets, we need the generic output folder

# Define the Excel file containing the tables
# ===========================================

input_excel_filename = 'D:/Users/Kirker/Documents/Git repositories/excel2latexviapython/Example/example_tables.xlsx'

# Select the directory/folder to save the resulting TeX files to

output_dir = 'D:/Users/Kirker/Documents/Git repositories/excel2latexviapython/Example/'

# Define user settings in a dictionary
# ====================================
#
# Current options:
#
#    booktabs: True/False
#        Defines if we should use the booktabs package functions to make prettier horizontal lines or standard hlines
#
# includetabular: True/False
#        Should the code include the tabular environment code around the table, or just return the table rows.    

usr_settings = {'booktabs':True, 'includetabular':True}

## Preamble


In [150]:
# Get and/or set the working directory
import re
import os
os.getcwd() # Get current working directory
#os.chdir() # Set current working directory

import openpyxl # Package for reading excel files (.xlsx) into Python

from itertools import compress

## Load data
Load in the Excel Workbook, and select the appropriate Work Sheet that contains the table to copy.

In [151]:
# read in excel workbook file
workbook = openpyxl.load_workbook(filename = input_excel_filename)
type(workbook)

openpyxl.workbook.workbook.Workbook

In [152]:
# Get the list of sheets in the workbook
workbook.get_sheet_names()

['simple_table', 'merged_cells', 'styled_table']

In [153]:
# Read in the worksheet from the workbook that contains teh table to replicate
#sheet = workbook[sheet_name]
#sheet

In [154]:
# Examin the table to get an idea of its dimensions.
#sheet.rows # See all cells in sheet 

In [155]:
# Check dimensions of the table
#print("number of columns = " + str(sheet.max_column))
#print("number of rows = " + str(sheet.max_row))

## Define functions
Subfunctions we are going to need to process the Excel file

In [156]:
def is_number(s):
    '''
    IS_NUMBER
    
    This function tells us if the object "s" is a number or not.
    
    This is useful for working out if the excel cell contains a number, and hence 
    whether we need to check how many d.p. to round to. It checks by trying to 
    convert the string to a float. If it fails, "s" is not a number
    
    Args:
        s: [string]
        
    Returns:
        True/False
    '''
    
    try:
        float(s)
        return True
    except ValueError:
        return False

In [157]:
def tupple2latexstring(row_tup, merge_list=[[],[],[]]):
    """ TUPPLE2LATEXSTRING
    
    This function converts a tupple of openpyxl CELLs into a single row string 
    of LaTeX code for inclusion in a table. It loops over each cell and appends 
    the appropriate text (respresenting the LaTeX code) to the string which it 
    returns at the end.
    
    
   Args:
        row_tup: [tupple] contains the openpyxl CELLs for a single row of the table
        
        
        merged_details_list


    Returns:
        A string of the row cells formatted in the LaTeX style.
        
    """
    
    
    num_elements = len(row_tup) # how many columns we have in the row
    
    
    str_out = "" # initilise the output string
    
    
    
    merge_start_cols = merge_list[0]
    merge_end_cols = merge_list[1]
    merge_match_det = merge_list[2]
    
    num_multi_in_row = len(merge_start_cols)
    
    colidx = 0
    multiidx = 0
    
    while colidx < num_elements: # for each column/cell
        
        # Check to see if the column/cell is part of a multicolumn/row
        
        if colidx in merge_start_cols :
            # Multicolumn/row
            
            
            value_string = merge_match_det[multiidx]
            
            colidx = merge_end_cols[multiidx]
            
            
            multiidx += 1
            
            
        else        :
            # Get the main text for that cell.

            #########
            # Step 1: Get the "value_string" giving the text displayed in the cell
            #########

            if row_tup[colidx].value == None:

                # In this case, the cell is empty, so 

                value_string = " " # Cell is empty of value

            else:   # Case when the cell contains something


                # Check the content of the cell:
                # The user might have used Excel to round off the number. 
                # If so, we need to apply the same rounding. So first check 
                # to see if the cell contains a number

                if is_number(row_tup[colidx].value):
                    # If it is a number, need to check if there is any rounding applied

                    if row_tup[colidx].number_format == 'General': 
                        # General indicates the user has not adjusted the number of d.p., so we can take just the raw number

                        value_string = str(row_tup[colidx].value)

                    else:
                        # User has rounded off the number in the cell to a specific number of d.p.

                        # work out how many d.p.s the user has rounded to. by splitting the string at 
                        # the decimal place, and counting the number of elements after the decimal

                        num_format_str =re.split('\.',row_tup[colidx].number_format) 

                        num_dps=len(num_format_str[1])

                        # Round the cell's value to that number of d.p. 
                        value_string = str(round(row_tup[colidx].value,num_dps))

                else:
                    # Cell contains no specific rounding choice by the user, so we can take just the straight value
                    value_string = str(row_tup[colidx].value)


            #########
            # Step 2: Apply formatting to the cell's value
            #########

            # The cell might have special formatting applied to the value inside it (e.g. bold text). 
            # Apply the LaTeX version of this formatting to the string    


            # Apply bold font if needed    
            if row_tup[colidx].font.__dict__['b']:
                value_string = "\\textbf{" + value_string + "}"

            # Apply italicize if needed
            if row_tup[colidx].font.__dict__['i']:
                value_string = "\\textit{" + value_string + "}"


        #########
        # Step 3: Now that we have to LaTeX code for that cell/column, append it to the string for the entire row.
        #########

        # Append formatted string for this cell to the string out
        str_out += value_string 


        # If this isnt the last element, add cell divider
        if colidx < num_elements-1:
            str_out += " \t & \t "

        colidx += 1
    
    # Now that we have looped over all elements, add on line ending code for the end of the row string
    str_out += " \\\ \n" 
    
    
    return str_out

In [158]:
def check_for_vline(col_tup,loc):
    """ 
    
    Look for vertical lines down the entire length of the column.
    
    We do this by looping over all the cells, and then counting how many of them have a vertical line in location "loc"
    
    Args:
        row_tup: [tupple] contains the openpyxl CELLs for a single row of the table


    Returns:
        True/False
    """    
    
    
    num_rows=len(col_tup) # Number of rows in the column
    
    ID = 0 # Initialise count
    
    
    for rownum in range(0, num_rows):  # For each row
        
        # Check to see if there is a border style in location "loc"
        if col_tup[rownum].border.__dict__[loc].border_style != None:
            
            # Add one to our count
            ID += 1
            
            
    # Check to see if every row has a border style in location "loc"
    if ID == num_rows:
        return(True)
    else:
        return(False)
    
    
    
# A test of the function:    
#out = check_for_vline(sheet.columns[2],'left')    
#print(out)

In [159]:
def create_cline_code(cell_has_rule,booktabs=False):
    """ 
    CREATE_CLINE_CODE
    
    Creates the code for horizontal lines that do not span the entire length of the table.
    
    
    Args:
        cell_has_rule: [list] whose elements are True/False for each cell indicating 
                            whether the horizontal rule includes this cell.
        
        booktabs=False: True/False. Should the code return code for the booktabs package or regular LaTeX?


    Returns:
        A string containing the code needed to draw the horizontal lines. E.g. "\cmidrule(r){1-4} \cmidrule(r){6-9} \n"
    """       
    
    # Initialize the output string based upon which table style we are doing
    
    if booktabs == True:
        str_out = '\\cmidrule(r)'
    else:
        str_out = '\\cline' 
    
    
   
    # Loop over each element of cell_has_rule to find where the lines start/stop
    
    
    num_column=len(cell_has_rule) # How many elements in the row
    
    # Create a flag to indicate whether we are starting a new cmidrule/cline or not.
    start_flag = 1 # we are looking for the next True
    
    
    for colind in range(0,num_column): # For each column/cell

        if (cell_has_rule[colind] == True) & (start_flag == 1):
            
            # We are initializing a new cmidrule/cline

            colnum = colind+1

            if booktabs == True:
                str_out +=  '\\cmidrule(r){'+ str(colnum) + '-'
            else:
                str_out +=  '\\cline{'+ str(colnum) + '-'

            start_flag = 0 # Turn off flag since now we are going to be looking for where this particular cline ends
            
            continue

        elif (cell_has_rule[colind] == False) & (start_flag == 1):

            # This is the case when we are searching for a new cline to begin, but there is some cells where it hasnt started yet
            str_out += '' # do nothing
            
            continue

        elif cell_has_rule[colind] == False: 
            
            # We have found a column/cell without a cline, so end the current open cline

            str_out += str(colind) + '} \t '

            start_flag = 1 # Turn flag back on so we are searching for a new cline start
            continue

        elif (cell_has_rule[colind] == True) & (colind==num_column-1):
        
            # If we get to the end of the table, and the column/cell still has a cline, end the cline.
            # last one is True
            
            str_out += str(num_column) + '} \t '
        
        else:
            # Code should never reach this here.
            
            str_out +' Err in create_cline_code'
            continue


    # End the line and return the string
    str_out += ' \n'

    return(str_out)

In [160]:
def create_horzrule_code(row_tup,loc,usr_settings,top_row = False, bottom_row = False):
    """ 
    CREATE_HORZRULE_CODE
    
    Creates the code for horizontal lines in a particular row of the table. Will return 
    a string of LaTeX code that is either a horizontal line spanning the entire width of 
    the table, or code to make the horizontal line(s) span only part of the table.
    
    
    Args:
        row_tup: [tuple] containing a row of the table.
        
        loc: [string] either 'top' or 'bottom' to indicate where (relative to this particular 
                row) we should check for any horizontal lines
        
        usr_settings: [dictionary] user settings - tells us to use booktabs code or not.
        
        

    Returns:
        A string containing the code needed to draw the horizontal line(s) for that particular row.
    """  
    
    
    '''
    Inputs:
    * row_tup : tupple containing the excel row
    
    * loc : String , either 'top' or 'bottom'
    
    * usr_settings : dirctionary, so we know if the user wants booktabs
    '''
    
    
    num_column=len(row_tup)
    
    
    #########
    # Step 1: Find which cells have horizontal rules
    #########
    
    # Contruct a list with True/False elements to indicate if the horizontal rule applies to that cell.
    
    cell_has_rule = [] # Pre-allocate list
    
    for colnum in range(0,num_column):
        
        if row_tup[colnum].border.__dict__[loc].border_style != None:
            cell_has_rule.append(True) 
        else:
            cell_has_rule.append(False)
    
    
    
    
    if sum(cell_has_rule) == 0: # If there are no rules and any cell, there is no line here, so return a blank string
        
        return('')
    
    else: # There exists some horizontal rule on at least part of the row, so return the appropriate LaTeX code  
    
    
    

        # If user has specified booktabs
        if usr_settings['booktabs'] == True:

            if sum(cell_has_rule) == num_column:

                return('\midrule \n')

            else:

                return(create_cline_code(cell_has_rule,booktabs=True))






        else:

            if sum(cell_has_rule) == num_column:
                return('\hline \n')

            else:

                return(create_cline_code(cell_has_rule,booktabs=False))

In [161]:
def get_merged_cells(sheet):
    
    
    start_row = []
    start_col = []
    
    end_row = []
    end_col = []
    
    latex_code = []
    

    if len(sheet.merged_cell_ranges) == 0:
        
        return([[],[],[],[],[]])
    
    
    
    for merge_ in sheet.merged_cell_ranges:  # For each merge in the sheet
        
        
        # Split the location string of the merge, and convert it it to an index number (e.g. "A3")
        merge_loc_str =re.split(':',merge_) 

        merge_cell_start_str = merge_loc_str[0]
        merge_cell_end_str = merge_loc_str[1]
        
        
        # convert string to col/row index numbers
        start_coord = openpyxl.utils.coordinate_to_tuple(merge_loc_str[0])
        end_coord = openpyxl.utils.coordinate_to_tuple(merge_loc_str[1])
        
        start_row.append(start_coord[0] - 1)
        start_col.append(start_coord[1] - 1)
        
        end_row.append(end_coord[0] - 1)
        end_col.append(end_coord[1] - 1)
        
        
        value_string = sheet[merge_loc_str[0]].value
        
        ###HERE###
        
        if sheet[merge_loc_str[0]].font.__dict__['b']:
            value_string = "\\textbf{" + value_string + "}"

        # Apply italicize if needed
        if sheet[merge_loc_str[0]].font.__dict__['i']:
            value_string = "\\textit{" + value_string + "}"
                
        # Get span of multicolumn        
        multi_col_length = end_coord[1] - start_coord[1] + 1
        
        # Get alignment
        halign =  sheet[merge_loc_str[0]].alignment.__dict__['horizontal'][0] # get the first letter
        
        
        latex_code.append('\multicolumn{' + str(multi_col_length) + '}{' + halign + '}{' +  value_string + '}')
              
       

        
        
    return([start_row, start_col, end_row, end_col, latex_code])
        
        
        
 

In [162]:
def pick_col_text_alignment(col_tup):
    """ 
    PICK_COL_TEXT_ALIGNMENT
    
    For a given column, choose the alignment (left, center, right) based 
    on the alignment choice of the majority of the cells
    
    
    Args:
        col_tup: [tuple] containing a column of the table.
        
    Returns:
        A string ('l'/'c'/'r') indicating the alignment to use
    """  
  
    
    max_column=len(col_tup)
    
    # Preallocate counters
    count_left = 0
    count_center = 0
    count_right = 0
    
    
    # Loop over each row, and count the alignment types
    for rn in range(0,max_column):
        
        # If the user doesnt speicify an alignment in Excel, we see the alignment 
        # choice as "None". So let us assign default values. If a number, align 
        # right, if not, align left.
        
        if col_tup[rn].alignment.__dict__['horizontal'] is None:
            
            # Check to see if the value is a number
            if col_tup[rn].value is None:
                align_val = 'ignore'
            elif is_number(col_tup[rn].value) :
                align_val = 'right'
            else:
                align_val = 'left'
                
        else:
            align_val = col_tup[rn].alignment.__dict__['horizontal']
        
        
        if align_val in ['left']:
            
            count_left += 1
            
        elif align_val in ['center']:
            
            count_center += 1
            
        elif align_val in ['right']:
            
            count_right += 1
            
            
    # Find the maximum, in the case of a tie, we break the tie by the order: L,C,R
    max_count = max([count_left, count_center, count_right])
    
    if count_left == max_count:
        
        return('l')
    
    elif count_center == max_count:
        
        return('c')
    
    elif count_right == max_count:
        
        return('r')
    

## Create LaTeX File
The main body of the code starts below. It reads in the excel file, loops over each worksheet in the workbook and writes a `.tex` file for each worksheet.

In [163]:
# Load in the Excel workbook/file
workbook = openpyxl.load_workbook(filename = input_excel_filename)


for sheet_name in workbook.get_sheet_names():  # Loop over every worksheet (tab) within the workbook
    
    #Get the worksheet 
    sheet = workbook[sheet_name]
    
    # Find any merged cells within this particular worksheet
    merged_details_list = get_merged_cells(sheet)
    

    # Create output file
    file = open(output_dir + sheet_name + '.tex', 'w') # 

    
    # If the user requested the booktabs options, add a reminder (as a LaTeX comment) to the top 
    # of the table that the user will need to load up the package in the preamble of their file.
    
    if usr_settings['booktabs']==True :
        
        file.write('% Note: make sure \\usepackage{booktabs} is included in the preamble \n')
    

    if usr_settings['includetabular']:
    ## Create Tabular Header

        col_align_str = "" # Preallocate string

        for colnum in range(0,sheet.max_column):

            if check_for_vline(sheet.columns[colnum],'left'):
                col_align_str += '|'

            col_align_str += pick_col_text_alignment(sheet.columns[colnum])

            if check_for_vline(sheet.columns[colnum],'right'):
                col_align_str += '|'

        begin_str = "\\begin{tabular}{" + str(col_align_str)  + "} \n"

        file.write(begin_str)




    for row_num in range(0,sheet.max_row):
        
        
        # Check to see if row contains any multicolumns/rows
        

        # Generate list of True/False values to see if they match the row
        elem_picker = [ True if item in [row_num] else False for item in merged_details_list[0] ]

        # Pick out the column number and mutlicolumn/row details corresponding to this row
        merge_start_cols = list(compress(merged_details_list[1], elem_picker))
        merge_end_cols   = list(compress(merged_details_list[3], elem_picker))
        merge_match_det  = list(compress(merged_details_list[4], elem_picker))
        



      
        
        #if check_for_hline(sheet.rows[row_num],'top') == True:
        #    
        #    if usr_settings['booktabs'] == True & row_num == 0:
        #        file.write("\top rule \n")
        #    elif usr_settings['booktabs'] == True:
        #        file.write("\midrule \n")
        #    else:
        #        file.write("\hline \n")

        hrule_str = create_horzrule_code( sheet.rows[row_num] , 'top' , usr_settings, top_row = False, bottom_row = False)
        
        
        if  (row_num == 0) & (usr_settings['booktabs']==True) :
            
            hrule_str = hrule_str.replace('\\midrule','\\toprule')
        
        file.write(hrule_str)
        
        
        #file.write(str(row_num)+"\t \\\ \n")
        file.write(tupple2latexstring(sheet.rows[row_num],[merge_start_cols, merge_end_cols, merge_match_det]))

        #if check_for_hline(sheet.rows[row_num],'bottom') == True:
        #    
        #    if usr_settings['booktabs'] == True & sheet.max_row == 0:
        #        file.write("\bottomrule \n")
        #    elif usr_settings['booktabs'] == True:
        #        file.write("\midrule \n")
        #    else:
        #        file.write("\hline \n")
        
        hrule_str = create_horzrule_code(sheet.rows[row_num] , 'bottom' , usr_settings,top_row = False, bottom_row = False)
        
        
        if  (row_num == sheet.max_row-1) & usr_settings['booktabs'] :
            
            hrule_str = hrule_str.replace('\\midrule','\\bottomrule')
            
        file.write(hrule_str)
            
    if usr_settings['includetabular']:
        # Close table environment
        file.write("\\end{tabular}")
        
        
        
    file.close() # Close off the current .tex file (completing the creation of the table code)